Install the libraries

In [1]:
pip install pandas sqlalchemy psycopg2-binary dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

from sqlalchemy import create_engine
from datetime import datetime, timedelta
import time

from dotenv import load_dotenv
import os
load_dotenv()
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB = os.getenv("POSTGRES_DB")


Gets the data

In [3]:
def get_titulos_tesouro_direto():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df = pd.read_csv(url, sep=';', decimal=',')
    df['Data_Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
    df['Data_Base'] = pd.to_datetime(df['Data Base'], dayfirst=True)
    multi_indice = pd.MultiIndex.from_frame(df.iloc[:,:3])
    df = df.set_index(multi_indice).iloc[:, 3:]
    return df

Loads the df

In [4]:
titles = get_titulos_tesouro_direto()

In [5]:
titles

Taxa Compra Manha  \
Tipo Titulo                            Data Vencimento Data Base                       
Tesouro Prefixado                      01/07/2007      30/08/2006              14.22   
                                       01/04/2007      30/08/2006              14.22   
                                       01/01/2009      30/08/2006              14.48   
                                       01/10/2006      30/08/2006              14.42   
                                       01/01/2007      30/08/2006              14.26   
Tesouro IPCA+ com Juros Semestrais     15/08/2008      30/08/2006               9.64   
                                       15/05/2007      30/08/2006               9.40   
                                       15/05/2009      30/08/2006               9.77   
                                       15/05/2045      30/08/2006               8.08   
                                       15/05/2035      30/08/2006               8.12   
...                                                                              ...   
Tesouro IGPM+ com Juros Semestrais     01/01/2031      20/01/2005               8.25   
                                       01/04/2021      20/01/2005               8.26   
                                       01/07/2017      20/01/2005               8.26   
                                       01/03/2011      20/01/2005               8.39   
                                       01/12/2005      20/01/2005              10.22   
                                       01/12/2006      20/01/2005               9.48   
                                       01/04/2008      20/01/2005               9.08   
                                       01/07/2005      20/01/2005              11.03   
Tesouro Prefixado com Juros Semestrais 01/01/2008      20/01/2005              17.78   
Tesouro Selic                          16/02/2005      19/01/2005               0.01   

                                                                   Taxa Venda Manha  \
Tipo Titulo                            Data Vencimento Data Base                      
Tesouro Prefixado                      01/07/2007      30/08/2006             14.26   
                                       01/04/2007      30/08/2006             14.26   
                                       01/01/2009      30/08/2006             14.54   
                                       01/10/2006      30/08/2006             14.45   
                                       01/01/2007      30/08/2006             14.29   
Tesouro IPCA+ com Juros Semestrais     15/08/2008      30/08/2006              9.66   
                                       15/05/2007      30/08/2006              9.42   
                                       15/05/2009      30/08/2006              9.81   
                                       15/05/2045      30/08/2006              8.18   
                                       15/05/2035      30/08/2006              8.22   
...                                                                             ...   
Tesouro IGPM+ com Juros Semestrais     01/01/2031      20/01/2005              8.35   
                                       01/04/2021      20/01/2005              8.34   
                                       01/07/2017      20/01/2005              8.34   
                                       01/03/2011      20/01/2005              8.45   
                                       01/12/2005      20/01/2005             10.24   
                                       01/12/2006      20/01/2005              9.50   
                                       01/04/2008      20/01/2005              9.12   
                                       01/07/2005      20/01/2005             11.05   
Tesouro Prefixado com Juros Semestrais 01/01/2008      20/01/2005             17.88   
Tesouro Selic                          16/02/2005      19/01/2005              0.03   

                                        

Gets the Prefixados

In [6]:
prefixado = titles.loc[('Tesouro Prefixado')].copy()
prefixado['Tipo'] = "PRE-FIXADOS"

In [7]:
prefixado

,,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,Data_Vencimento,Data_Base,Tipo
Data Vencimento,Data Base,,,,,,,,
01/07/2007,30/08/2006,14.22,14.26,897.01,896.75,896.28,2007-07-01,2006-08-30,PRE-FIXADOS
01/04/2007,30/08/2006,14.22,14.26,926.83,926.65,926.16,2007-04-01,2006-08-30,PRE-FIXADOS
01/01/2009,30/08/2006,14.48,14.54,730.18,729.29,728.89,2009-01-01,2006-08-30,PRE-FIXADOS
01/10/2006,30/08/2006,14.42,14.45,988.83,988.81,988.28,2006-10-01,2006-08-30,PRE-FIXADOS
01/01/2007,30/08/2006,14.26,14.29,957.54,957.46,956.96,2007-01-01,2006-08-30,PRE-FIXADOS
01/01/2008,29/08/2006,14.36,14.40,837.51,837.13,836.68,2008-01-01,2006-08-29,PRE-FIXADOS
01/04/2008,29/08/2006,14.39,14.44,810.42,809.86,809.43,2008-04-01,2006-08-29,PRE-FIXADOS
01/07/2008,29/08/2006,14.44,14.49,783.43,782.81,782.39,2008-07-01,2006-08-29,PRE-FIXADOS
01/10/2007,29/08/2006,14.33,14.37,865.87,865.54,865.08,2007-10-01,2006-08-29,PRE-FIXADOS


In [8]:
for i, row in prefixado.iterrows():
    ifor_val = datetime.now() - timedelta(hours=1, minutes=0)
    prefixado.at[i,'dt_update'] = ifor_val
    time.sleep(1/10000)

In [9]:
prefixado

,,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,Data_Vencimento,Data_Base,Tipo,dt_update
Data Vencimento,Data Base,,,,,,,,,
01/07/2007,30/08/2006,14.22,14.26,897.01,896.75,896.28,2007-07-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.634472
01/04/2007,30/08/2006,14.22,14.26,926.83,926.65,926.16,2007-04-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.637382
01/01/2009,30/08/2006,14.48,14.54,730.18,729.29,728.89,2009-01-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.637949
01/10/2006,30/08/2006,14.42,14.45,988.83,988.81,988.28,2006-10-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.638519
01/01/2007,30/08/2006,14.26,14.29,957.54,957.46,956.96,2007-01-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.639170
01/01/2008,29/08/2006,14.36,14.40,837.51,837.13,836.68,2008-01-01,2006-08-29,PRE-FIXADOS,2025-05-22 13:07:19.639932
01/04/2008,29/08/2006,14.39,14.44,810.42,809.86,809.43,2008-04-01,2006-08-29,PRE-FIXADOS,2025-05-22 13:07:19.640675
01/07/2008,29/08/2006,14.44,14.49,783.43,782.81,782.39,2008-07-01,2006-08-29,PRE-FIXADOS,2025-05-22 13:07:19.641493
01/10/2007,29/08/2006,14.33,14.37,865.87,865.54,865.08,2007-10-01,2006-08-29,PRE-FIXADOS,2025-05-22 13:07:19.642301


Get the IPCAs

In [10]:
ipca = titles.loc[('Tesouro IPCA+')].copy()
ipca['Tipo'] = "IPCA"

In [11]:
for i, row in ipca.iterrows():
    ifor_val = datetime.now() - timedelta(hours=1, minutes=0)
    ipca.at[i,'dt_update'] = ifor_val
    time.sleep(1/10000)

In [12]:
ipca

,,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,Data_Vencimento,Data_Base,Tipo,dt_update
Data Vencimento,Data Base,,,,,,,,,
15/05/2015,30/08/2006,9.03,9.09,755.66,752.06,751.75,2015-05-15,2006-08-30,IPCA,2025-05-22 13:07:29.932981
15/08/2024,30/08/2006,7.90,7.98,410.47,405.07,404.92,2024-08-15,2006-08-30,IPCA,2025-05-22 13:07:29.935374
15/05/2015,29/08/2006,9.02,9.08,755.95,752.35,752.04,2015-05-15,2006-08-29,IPCA,2025-05-22 13:07:29.935914
15/08/2024,29/08/2006,7.87,7.95,412.37,406.94,406.79,2024-08-15,2006-08-29,IPCA,2025-05-22 13:07:29.936453
15/05/2015,23/08/2006,8.94,9.00,759.65,756.02,755.69,2015-05-15,2006-08-23,IPCA,2025-05-22 13:07:29.937031
15/08/2024,23/08/2006,7.82,7.90,415.25,409.78,409.61,2024-08-15,2006-08-23,IPCA,2025-05-22 13:07:29.937612
15/05/2015,22/08/2006,8.98,9.04,756.90,753.29,752.96,2015-05-15,2006-08-22,IPCA,2025-05-22 13:07:29.938234
15/08/2024,22/08/2006,7.79,7.87,417.17,411.66,411.50,2024-08-15,2006-08-22,IPCA,2025-05-22 13:07:29.938858
15/05/2015,21/08/2006,9.00,9.06,755.37,751.76,751.43,2015-05-15,2006-08-21,IPCA,2025-05-22 13:07:29.939501


In [13]:
#frames = [prefixado, ipca]
#todos = pd.concat(frames)
ipca = ipca.rename(columns={"Data Vencimento": "DataVencimento", "Data Base": "Data_Base", "Taxa Compra Manha": "CompraManha", "Taxa Venda Manha": "VendaManha", "PU Compra Manha": "PUCompraManha", "PU Venda Manha": "PUVendaManha",  "PU Base Manha": "PUBaseManha"})
prefixado = prefixado.rename(columns={"Data Vencimento": "DataVencimento", "Data Base": "Data_Base", "Taxa Compra Manha": "CompraManha", "Taxa Venda Manha": "VendaManha", "PU Compra Manha": "PUCompraManha", "PU Venda Manha": "PUVendaManha",  "PU Base Manha": "PUBaseManha"})

In [14]:
ipca

,,CompraManha,VendaManha,PUCompraManha,PUVendaManha,PUBaseManha,Data_Vencimento,Data_Base,Tipo,dt_update
Data Vencimento,Data Base,,,,,,,,,
15/05/2015,30/08/2006,9.03,9.09,755.66,752.06,751.75,2015-05-15,2006-08-30,IPCA,2025-05-22 13:07:29.932981
15/08/2024,30/08/2006,7.90,7.98,410.47,405.07,404.92,2024-08-15,2006-08-30,IPCA,2025-05-22 13:07:29.935374
15/05/2015,29/08/2006,9.02,9.08,755.95,752.35,752.04,2015-05-15,2006-08-29,IPCA,2025-05-22 13:07:29.935914
15/08/2024,29/08/2006,7.87,7.95,412.37,406.94,406.79,2024-08-15,2006-08-29,IPCA,2025-05-22 13:07:29.936453
15/05/2015,23/08/2006,8.94,9.00,759.65,756.02,755.69,2015-05-15,2006-08-23,IPCA,2025-05-22 13:07:29.937031
15/08/2024,23/08/2006,7.82,7.90,415.25,409.78,409.61,2024-08-15,2006-08-23,IPCA,2025-05-22 13:07:29.937612
15/05/2015,22/08/2006,8.98,9.04,756.90,753.29,752.96,2015-05-15,2006-08-22,IPCA,2025-05-22 13:07:29.938234
15/08/2024,22/08/2006,7.79,7.87,417.17,411.66,411.50,2024-08-15,2006-08-22,IPCA,2025-05-22 13:07:29.938858
15/05/2015,21/08/2006,9.00,9.06,755.37,751.76,751.43,2015-05-15,2006-08-21,IPCA,2025-05-22 13:07:29.939501


Saves in the database

In [15]:
connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

In [16]:
engine = create_engine(connection_string)

In [17]:
ipca.to_sql("postg_ipca", con=engine, if_exists="replace", index=False, schema="public")
prefixado.to_sql("postg_pre", con=engine, if_exists="replace", index=False, schema="public")

300

In [18]:
query = "SELECT * FROM public.postg_pre"
df = pd.read_sql(query, con=engine)

# check pre
df.head()

,CompraManha,VendaManha,PUCompraManha,PUVendaManha,PUBaseManha,Data_Vencimento,Data_Base,Tipo,dt_update
0,14.22,14.26,897.01,896.75,896.28,2007-07-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.634472
1,14.22,14.26,926.83,926.65,926.16,2007-04-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.637382
2,14.48,14.54,730.18,729.29,728.89,2009-01-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.637949
3,14.42,14.45,988.83,988.81,988.28,2006-10-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.638519
4,14.26,14.29,957.54,957.46,956.96,2007-01-01,2006-08-30,PRE-FIXADOS,2025-05-22 13:07:19.639170


In [19]:
query = "SELECT * FROM public.postg_ipca"
df = pd.read_sql(query, con=engine)

# check ipca
df.head()

,CompraManha,VendaManha,PUCompraManha,PUVendaManha,PUBaseManha,Data_Vencimento,Data_Base,Tipo,dt_update
0,9.03,9.09,755.66,752.06,751.75,2015-05-15,2006-08-30,IPCA,2025-05-22 13:07:29.932981
1,7.90,7.98,410.47,405.07,404.92,2024-08-15,2006-08-30,IPCA,2025-05-22 13:07:29.935374
2,9.02,9.08,755.95,752.35,752.04,2015-05-15,2006-08-29,IPCA,2025-05-22 13:07:29.935914
3,7.87,7.95,412.37,406.94,406.79,2024-08-15,2006-08-29,IPCA,2025-05-22 13:07:29.936453
4,8.94,9.00,759.65,756.02,755.69,2015-05-15,2006-08-23,IPCA,2025-05-22 13:07:29.937031
